In [ ]:
from packages import *
from custom_tf_layers import *
from models import *
from utils import *
from pretrain_ae import pretrain_autoencoder
from train import train

In [ ]:
#load formated data
with open("weekwise_encoded_variantlength.pickle", "rb") as file:
    med = pickle.load(file)
    
for patient in med.keys():
    for week in med[patient].keys():
        tmp = list(set(med[patient][week]))
        med[patient][week] = [week] + tmp
    med[patient] = list(med[patient].values())
    
med = list(med.values())

med = tf.ragged.constant(med)
med.shape

# Autoencoder pretraining

In [ ]:
#import autoencoder model architecture
hercules = define_hercules(1569, 1571, 64)
plot_model(hercules)

In [ ]:
#pretrain the autoencoder
BUFFER_SIZE = 16000
BATCH_SIZE = 64
N_EPOCHS = 300

pretrain_autoencoder(hercules, ehr_dataset=med, BUFFER_SIZE=BUFFER_SIZE, 
                     BATCH_SIZE=BATCH_SIZE, N_EPOCHS=N_EPOCHS)

# GAN training

In [ ]:
# define generator
input_layer = layers.Input(shape=(None, 64), ragged=True)
pretrained_decoder_layer = K.function([hercules.layers[5].input], [hercules.layers[6].output])(input_layer)[0]
pretrained_decoder = models.Model(inputs= [input_layer], outputs= [pretrained_decoder_layer])
generator = define_generator(pretrained_decoder)
plot_model(generator)

In [ ]:
# define discriminator
preprocess_model = define_preprocess(NUM_CODE=1569)
discriminator = define_discriminator(NUM_CODE=1569, MAX_TIMESTEPS=257, EMBEDIM=64)
plot_model(discriminator)

In [ ]:
#train the gan
train(generator, discriminator, preprocess_model, med, EPOCHS=300)